# Cyclonic Cartography: Visualizing Tornado Trends

## Part 1: Database and Jupyter Notebook Set Up

Import the storm data retrieved from the online source from your Terminal. The database will be named `storm_data` and the collection `tornadoes`.

Imported the dataset with: 
mongoimport --type csv -d storm_data -c tornadoes --headerline --drop all_storms.csv

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
# the list of databases should include the newly imported databased named storm_data
print(mongo.list_database_names())

['admin', 'class_db', 'config', 'epa', 'fruits_db', 'garden_db', 'local', 'met', 'petsitly_marketing', 'storm_data', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['storm_data']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['tornadoes']


In [6]:
# review a document in the establishments collection
pprint(db.tornadoes.find_one())

{'BEGIN_AZIMUTH': 'SE',
 'BEGIN_DATE': '1/25/2021',
 'BEGIN_LAT': 33.5965,
 'BEGIN_LOCATION': 'WALKER CHAPEL',
 'BEGIN_LON': -86.8256,
 'BEGIN_RANGE': 0,
 'BEGIN_TIME': 2240,
 'CZ_NAME_STR': 'JEFFERSON CO.',
 'CZ_TIMEZONE': 'CST',
 'DAMAGE_CROPS_NUM': 0,
 'DAMAGE_PROPERTY_NUM': 0,
 'DEATHS_DIRECT': 1,
 'END_AZIMUTH': 'WNW',
 'END_DATE': '1/25/2021',
 'END_LAT': 33.6598,
 'END_LOCATION': 'CHALKVILLE',
 'END_LON': -86.6678,
 'END_RANGE': 1,
 'END_TIME': 2255,
 'EPISODE_ID': 155525,
 'EVENT_ID': 938002,
 'EVENT_TYPE': 'Tornado',
 'INJURIES_DIRECT': 30,
 'STATE_ABBR': 'AL',
 'TOR_F_SCALE': 'EF3',
 'TOR_LENGTH': 10.4,
 'TOR_WIDTH': 900,
 '_id': ObjectId('65dd5d4943dde26c5545321c')}


In [7]:
# assign the collection to a variable
tornadoes = db['tornadoes']

In [8]:
# find and replace all F3 values with EF3 and F4 values with EF4

tornadoes.update_many({'TOR_F_SCALE': 'F3'}, {'$set': {'TOR_F_SCALE': 'EF3'}})
tornadoes.update_many({'TOR_F_SCALE': 'F4'}, {'$set': {'TOR_F_SCALE': 'EF4'}})

In [9]:
# find and replace time zones with digits so they are all matching: CST, EST, MST
tornadoes.update_many({'CZ_TIMEZONE': 'CST-6'}, {'$set': {'CZ_TIMEZONE': 'CST'}})
tornadoes.update_many({'CZ_TIMEZONE': 'EST-5'}, {'$set': {'CZ_TIMEZONE': 'EST'}})
tornadoes.update_many({'CZ_TIMEZONE': 'MST-7'}, {'$set': {'CZ_TIMEZONE': 'MST'}})

## Part 2: Create DataFrames for Charting

### Create a table showing the deaths and injuries grouped by EF indicator

In [10]:
# Create a dataframe of the full data set
query = {'EVENT_TYPE': 'Tornado'}

results = list(tornadoes.find(query))

tornado_df = pd.json_normalize(results)

# print first 10 rows to check
tornado_df.head(10)

,_id,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,BEGIN_AZIMUTH,END_RANGE,END_AZIMUTH,END_LOCATION,END_DATE,END_TIME,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
0,65dd5d4943dde26c5545321c,938002,JEFFERSON CO.,WALKER CHAPEL,1/25/2021,2240,Tornado,EF3,1,30,...,SE,1,WNW,CHALKVILLE,1/25/2021,2255,33.5965,-86.8256,33.6598,-86.6678
1,65dd5d4943dde26c5545321d,948201,HALE CO.,GREENSBORO MUNI ARPT,3/25/2021,1526,Tornado,EF3,0,6,...,W,2,S,HOGGLESVILLE,3/25/2021,1547,32.6892,-87.7387,32.814,-87.473
2,65dd5d4943dde26c5545321e,946864,HALE CO.,HAVANA,3/25/2021,1116,Tornado,EF3,0,0,...,NE,6,NE,PHIPPS,3/25/2021,1129,32.9386,-87.5753,33.004,-87.421
3,65dd5d4943dde26c5545321f,948202,PERRY CO.,MORGAN SPGS,3/25/2021,1547,Tornado,EF3,0,0,...,NW,5,NW,ELLARDS,3/25/2021,1557,32.814,-87.473,32.864,-87.319
4,65dd5d4943dde26c55453220,946870,CALHOUN CO.,BOILING SPGS,3/25/2021,1331,Tornado,EF3,6,10,...,WSW,2,WSW,LADIGA,3/25/2021,1418,33.7026,-86.1053,33.962,-85.634
5,65dd5d4943dde26c55453221,948219,BIBB CO.,PONDVILLE,3/25/2021,1557,Tornado,EF3,0,5,...,S,3,E,BRIERFIELD,3/25/2021,1631,32.864,-87.319,33.027,-86.877
6,65dd5d4943dde26c55453222,945569,BRUNSWICK CO.,GRISSETTOWN,2/15/2021,2334,Tornado,EF3,3,10,...,SSW,3,E,CAMP BRANCH,2/16/2021,2,33.9118,-78.5096,34.1774,-78.3008
7,65dd5d4943dde26c55453223,954213,KNOX CO.,TRUSCOTT,4/27/2021,1449,Tornado,EF3,0,0,...,W,3,NNE,TRUSCOTT,4/27/2021,1505,33.751,-99.921,33.791,-99.788
8,65dd5d4943dde26c55453224,954642,ST. LANDRY PAR.,WAXIA,4/10/2021,106,Tornado,EF3,1,7,...,WSW,2,NNE,DANKS,4/10/2021,118,30.666,-91.9586,30.7499,-91.8492
9,65dd5d4943dde26c55453225,960282,COOK CO.,BURR RIDGE,6/20/2021,2221,Tornado,EF3,0,0,...,SSW,0,SSW,WILLOW SPRINGS,6/20/2021,2225,41.7374,-87.9151,41.7368,-87.8612


In [11]:
# make a dataframe showing only EF indicator and total number of direct deaths and direct injuries

deathInj_df = tornado_df[['TOR_F_SCALE', 'DEATHS_DIRECT', 'INJURIES_DIRECT']]

#print 5 rows to check
deathInj_df.head()




,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT
0,EF3,1,30
1,EF3,0,6
2,EF3,0,0
3,EF3,0,0
4,EF3,6,10


In [12]:
# group by EF scale
deathInj_grouped = deathInj_df.groupby(['TOR_F_SCALE'])

# find sums for deaths and injuries
deaths = deathInj_grouped['DEATHS_DIRECT'].sum()
injuries = deathInj_grouped['INJURIES_DIRECT'].sum()

# create new dataframe using sums
deathInj_counts_df = pd.DataFrame({'Deaths': deaths, 'Injuries': injuries})

deathInj_counts_df

,Deaths,Injuries
TOR_F_SCALE,,
EF3,603,7600
EF4,495,6226
EF5,312,1725


### Create a table showing count of tornadoes by time zone (by EF indicator)

In [13]:
# create new df from tornado df with only columns we want
timezone_df = tornado_df[['CZ_TIMEZONE', 'STATE_ABBR', 'TOR_F_SCALE', 'EVENT_ID']]

timezone_df.head()


,CZ_TIMEZONE,STATE_ABBR,TOR_F_SCALE,EVENT_ID
0,CST,AL,EF3,938002
1,CST,AL,EF3,948201
2,CST,AL,EF3,946864
3,CST,AL,EF3,948202
4,CST,AL,EF3,946870


In [14]:
# group by timezone
time_grouped = timezone_df.groupby(['CZ_TIMEZONE', 'TOR_F_SCALE'])

# find counts of each EF indicator
event_count = time_grouped['EVENT_ID'].count()

# new dataframe showing counts for timezone/EF
zone_df = pd.DataFrame({'Count': event_count})

zone_df

Count
CZ_TIMEZONE TOR_F_SCALE       
CST         EF3            635
            EF4            152
            EF5             14
EST         EF3            185
            EF4             21
MST         EF3             18

### Create a table showing count of tornadoes by state (by EF indicator)

In [15]:
# create new grouped by from timezone showing the state
state_grouped = timezone_df.groupby(['STATE_ABBR', 'TOR_F_SCALE'])

# find counts
ef_counts = state_grouped['EVENT_ID'].count()

#new dataframe showing counts for the state/EF
state_df = pd.DataFrame({'Count': ef_counts})

state_df

Count
STATE_ABBR TOR_F_SCALE       
AL         EF3             69
           EF4             24
           EF5              4
AR         EF3             44
           EF4              4
...                       ...
VA         EF3             13
           EF4              1
WI         EF3             15
WV         EF3              2
WY         EF3              3

[64 rows x 1 columns]

In [16]:
# create dataframe showing count of events by state
states_only_df = tornado_df[['STATE_ABBR', 'EVENT_ID']]

states_only_df.head()

,STATE_ABBR,EVENT_ID
0,AL,938002
1,AL,948201
2,AL,946864
3,AL,948202
4,AL,946870


In [40]:
# group by state and complete a count, then display as a new dataframe

state_group = states_only_df.groupby(['STATE_ABBR'])

storm_count = state_group['EVENT_ID'].count()

state_counts_df = pd.DataFrame({'Count': storm_count})

state_counts_df

,Count
STATE_ABBR,
AL,97
AR,48
AZ,2
CO,8
DE,1
FL,8
GA,58
IA,29
IL,41


### Create a table showing count of tornadoes by month for each state (include time zone and EF indicator)

In [17]:
# check tornado_df info

tornado_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   _id                  1025 non-null   object 
 1   EVENT_ID             1025 non-null   int64  
 2   CZ_NAME_STR          1025 non-null   object 
 3   BEGIN_LOCATION       1025 non-null   object 
 4   BEGIN_DATE           1025 non-null   object 
 5   BEGIN_TIME           1025 non-null   int64  
 6   EVENT_TYPE           1025 non-null   object 
 7   TOR_F_SCALE          1025 non-null   object 
 8   DEATHS_DIRECT        1025 non-null   int64  
 9   INJURIES_DIRECT      1025 non-null   int64  
 10  DAMAGE_PROPERTY_NUM  1025 non-null   int64  
 11  DAMAGE_CROPS_NUM     1025 non-null   int64  
 12  STATE_ABBR           1025 non-null   object 
 13  CZ_TIMEZONE          1025 non-null   object 
 14  EPISODE_ID           1025 non-null   int64  
 15  TOR_LENGTH           1025 non-null   f

In [18]:
# begin_date is an object; change to date/time
tornado_df['BEGIN_DATE'] = pd.to_datetime(tornado_df['BEGIN_DATE'])

#check it
tornado_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   _id                  1025 non-null   object        
 1   EVENT_ID             1025 non-null   int64         
 2   CZ_NAME_STR          1025 non-null   object        
 3   BEGIN_LOCATION       1025 non-null   object        
 4   BEGIN_DATE           1025 non-null   datetime64[ns]
 5   BEGIN_TIME           1025 non-null   int64         
 6   EVENT_TYPE           1025 non-null   object        
 7   TOR_F_SCALE          1025 non-null   object        
 8   DEATHS_DIRECT        1025 non-null   int64         
 9   INJURIES_DIRECT      1025 non-null   int64         
 10  DAMAGE_PROPERTY_NUM  1025 non-null   int64         
 11  DAMAGE_CROPS_NUM     1025 non-null   int64         
 12  STATE_ABBR           1025 non-null   object        
 13  CZ_TIMEZONE          1025 non-nul

In [19]:
# extract the month from the date/time into a new column
tornado_df['MONTH'] = tornado_df['BEGIN_DATE'].dt.month

tornado_df.head()

,_id,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_RANGE,END_AZIMUTH,END_LOCATION,END_DATE,END_TIME,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,MONTH
0,65dd5d4943dde26c5545321c,938002,JEFFERSON CO.,WALKER CHAPEL,2021-01-25,2240,Tornado,EF3,1,30,...,1,WNW,CHALKVILLE,1/25/2021,2255,33.5965,-86.8256,33.6598,-86.6678,1
1,65dd5d4943dde26c5545321d,948201,HALE CO.,GREENSBORO MUNI ARPT,2021-03-25,1526,Tornado,EF3,0,6,...,2,S,HOGGLESVILLE,3/25/2021,1547,32.6892,-87.7387,32.814,-87.473,3
2,65dd5d4943dde26c5545321e,946864,HALE CO.,HAVANA,2021-03-25,1116,Tornado,EF3,0,0,...,6,NE,PHIPPS,3/25/2021,1129,32.9386,-87.5753,33.004,-87.421,3
3,65dd5d4943dde26c5545321f,948202,PERRY CO.,MORGAN SPGS,2021-03-25,1547,Tornado,EF3,0,0,...,5,NW,ELLARDS,3/25/2021,1557,32.814,-87.473,32.864,-87.319,3
4,65dd5d4943dde26c55453220,946870,CALHOUN CO.,BOILING SPGS,2021-03-25,1331,Tornado,EF3,6,10,...,2,WSW,LADIGA,3/25/2021,1418,33.7026,-86.1053,33.962,-85.634,3


In [20]:
# df grouped by timezone showing counts by month by ef
month_df = tornado_df[['MONTH', 'CZ_TIMEZONE', 'TOR_F_SCALE', 'EVENT_ID']]

month_df.head()


,MONTH,CZ_TIMEZONE,TOR_F_SCALE,EVENT_ID
0,1,CST,EF3,938002
1,3,CST,EF3,948201
2,3,CST,EF3,946864
3,3,CST,EF3,948202
4,3,CST,EF3,946870


In [24]:
month_grouped = month_df.groupby(['MONTH', 'CZ_TIMEZONE'])


month_grouped.count()

TOR_F_SCALE  EVENT_ID
MONTH CZ_TIMEZONE                       
1     CST                   29        29
      EST                   12        12
2     CST                   50        50
      EST                   16        16
3     CST                   73        73
      EST                   48        48
      MST                    1         1
4     CST                  205       205
      EST                   55        55
5     CST                  200       200
      EST                   23        23
      MST                    2         2
6     CST                   68        68
      EST                    6         6
      MST                    8         8
7     CST                   15        15
      EST                    8         8
      MST                    4         4
8     CST                   21        21
      EST                    5         5
      MST                    1         1
9     CST                    6         6
      EST                   11        11
10    CST                   16        16
      EST                    4         4
      MST                    2         2
11    CST                   62        62
      EST                   14        14
12    CST                   56        56
      EST                    4         4